In [34]:
from typing import List

# DP + 状态压缩： TLE
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        # 每一个工作都对一个工人来讲，都可以做，或者不做
        n = len(jobs)
        dp = [[float('inf')] * (1 << n) for _ in range(k + 1)]
        times = [0] * (1 << n)
        for state in range(1 << n):
            sum_time = 0
            for i in range(n):
                if ((state >> i) & 1) == 1:
                    sum_time += jobs[i]
            times[state] = sum_time
            
        dp[0][0] = 0
        for i in range(1, k+1):
            for state in range(1 << n):
                sub_state = state
                while sub_state > 0:
                    dp[i][state] = min(dp[i][state], max(dp[i-1][state-sub_state], times[sub_state]))
                    sub_state = (sub_state - 1) & state
        return dp[-1][-1]

In [56]:
from typing import List
from functools import lru_cache

# 二分搜索 + 状态压缩 TLE
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        # 每一个工作都对一个工人来讲，都可以做，或者不做
        # @lru_cache(None)
        def dfs(state, th, m): # 给第m个人，分配工作，并且这个工作时间不能超过th
            if state == 0:
                return True
            if m == k:
                return False
            subset = state
            while subset > 0:
                if times[subset] <= th and dfs(state - subset, th, m+1):
                    return True
                subset = (subset - 1) & state
            return False
            
        n = len(jobs)
        dp = [[float('inf')] * (1 << n) for _ in range(k + 1)]
        times = [0] * (1 << n)
        for state in range(1 << n):
            sum_time = 0
            for i in range(n):
                if ((state >> i) & 1) == 1:
                    sum_time += jobs[i]
            times[state] = sum_time
        
        left, right = 1, sum(jobs)
        while left < right:
            mid = left + (right - left) // 2
            if dfs((1<<n)-1, mid, 0):
                right = mid
            else:
                left = mid + 1
        return left

In [57]:
solution = Solution()
solution.minimumTimeRequired(jobs = [1,2,4,7,8], k = 2)

11

'0b100'

In [68]:
from typing import List
from functools import lru_cache

# DFS + 二分搜索 + 状态压缩 TLE
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        def dfs(workers, th, idx):
            if idx == len(jobs): # 所有的任务都分配完成
                return True
            
            faliure = {} # 失败的案例有哪些
            for j in range(k):
                if workers[j] + jobs[idx] >  th:
                    continue
                    
                # 如果当前工作超过了上线，那么肯定不能添加
                if workers[j] in faliure and jobs[idx] >= faliure[workers[j]]:
                    continue
                
                workers[j] += jobs[idx]
                if dfs(workers, th, idx + 1):
                    return True
                workers[j] -= jobs[idx]
                
                # 记录了workers[j] 在 阈值为th下的工作上限是多少
                if workers[j] in faliure:
                    faliure[workers[j]] = min(jobs[idx], faliure[workers[j]])
                else:
                    faliure[workers[j]] = jobs[idx]
            return False
        
        jobs.sort(reverse=True)
        left, right = 1, sum(jobs)
        while left < right:
            workers = [0] * k # 表示每个人分配的工作时长有多少
            mid = left + (right - left) // 2
            if dfs(workers, mid, 0): # 其中 0 表示分配第 idx=0 的工作给工人
                right = mid
            else:
                left = mid + 1
        return left

In [69]:
solution = Solution()
solution.minimumTimeRequired(jobs = [1,2,4,7,8], k = 2)

11

In [82]:
from typing import List
from functools import lru_cache

# DFS + 二分搜索
class Solution:
    def minimumTimeRequired(self, jobs: List[int], k: int) -> int:
        def dfs(workers, th, idx):
            if idx == len(jobs):
                return True
            
            faliure = {}
            for i in range(k):
                # 不能有一个工人的最大工作时间大于 threshold
                if jobs[idx] + workers[i] > th:
                    continue
                # 如果当前工作超过了上限，那么肯定不能添加
                if workers[i] in faliure and jobs[idx] >= faliure[workers[i]]:
                    continue
                    
                workers[i] += jobs[idx]
                if dfs(workers, th, idx+1):
                    return True
                workers[i] -= jobs[idx]
                
                if workers[i] in faliure:
                    faliure[workers[i]] = min(faliure[workers[i]], jobs[idx])
                else:
                    faliure[workers[i]] = jobs[idx]
            return False
        
        jobs.sort(reverse=True)
        left, right = 1, sum(jobs)
        while left < right:
            workers = [0] * k
            mid = left + (right - left) // 2
            if dfs(workers, mid, 0):
                right = mid
            else:
                left = mid + 1
        return left

In [83]:
solution = Solution()
solution.minimumTimeRequired(jobs = [1,2,4,7,8], k = 2)

11